In [1]:
import requests as req
import re
from db import DB

In [2]:
ENT_P10 = 'https://www.gsmarena.com/huawei_p10-reviews-8514.php'
ENT_P10_PAT = 'https://www.gsmarena.com/huawei_p10-reviews-8514p{}.php'
ENT_SS8_PAT = 'https://www.gsmarena.com/samsung_galaxy_s8-reviews-8161p{}.php'
ENT_IP7P_PAT = ''
ENT_PIXEL_PAT = ''

In [3]:
res = req.get(ENT_P10)

In [18]:
rePager = re.compile(r'id="user-pages"([\s\S]*?)<\/div>')
rePageNo = re.compile(r'<a href=".*?(\d+)\.php"')
reCmt = re.compile(r'<p class="uopin">([\s\S]*?)<\/p>')

In [7]:
pager = rePager.findall(res.text)

In [10]:
rePageNo.findall(pager[0])

['2', '3', '36', '37', '2']

In [29]:
def process_comment(html):
    comments = reCmt.findall(html)
    result = []
    for cmt in comments:
        s = re.sub(r'<span.*?>[\s\S]*?<\/span>', '', cmt)
        s = re.sub(r'<a.*?>.*?<\/a>', '', s)
        s = re.sub(r'<br>', '.', s)
        
        result.append({
            'cmt': s,
        })
    return result

In [15]:
def process_ent(url):
    res = req.get(url)
    pager = rePager.findall(res.text)
    return int(rePageNo.findall(pager[0])[-2])

In [16]:
pageno = process_ent(ENT_P10)

In [38]:
pageno = 222

In [39]:
def process_product(pageno, pat, product):
    for page in range(1, pageno+1):
        url = pat.format(page)
        print(url)
        res = req.get(url)
        
        comments = process_comment(res.text)
        for i, item in enumerate(comments):
            item['product'] = product
            item['source'] = 'GSMARENA'
            item['kind'] = 'User_Review'
        DB.insert_many(comments)

In [44]:
process_product(pageno, ENT_SS8_PAT, 'S8')

https://www.gsmarena.com/samsung_galaxy_s8-reviews-8161p1.php


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))